In [1]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import (
    FunctionTransformer,
    PowerTransformer,
    MinMaxScaler,
    OneHotEncoder,
    Binarizer
)
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


import mlflow
import mlflow.sklearn


In [2]:
df_train = pd.read_csv('../data/raw/actuarial-loss-estimation/train.csv', parse_dates=['DateTimeOfAccident', 'DateReported'], index_col=0)

In [3]:
df_train['YearAccident'] = df_train['DateTimeOfAccident'].dt.year
df_train['DaysToReport'] = (df_train['DateReported'] - df_train['DateTimeOfAccident']).dt.days + 1 # no zero values


In [4]:
log_transformer = FunctionTransformer(np.log, validate=False)

# gender to bool 
def gender_to_bool(gender_column):
    """
    - Fill missing values with 'M'
    - Replace 'U' with 'M'
    - Return a boolean-ish column: 1 for 'M', 0 otherwise
    """
    g = pd.Series(gender_column.squeeze(), dtype=str).fillna('M').replace('U', 'M')
    is_male = (g == 'M').astype(int)
    return is_male.values.reshape(-1, 1)

gender_transformer = FunctionTransformer(gender_to_bool, validate=False)

# hours worked per week -> to buckets -> to one_hot
def bucket_hours_worked(dtt_array):
    return pd.cut(
        dtt_array.squeeze(), 
        bins=[-np.inf, 37, 41, np.inf],
        labels=["<=37", "37-41", ">41"]
    ).astype(str).values.reshape(-1, 1)

hours_worked_bucketer = FunctionTransformer(bucket_hours_worked, validate=False)
hours_worked_encoder = OneHotEncoder(drop='first')
hours_worked_pipeline = Pipeline([
    ('bucketizer', hours_worked_bucketer),
    ('encoder', hours_worked_encoder)
])

# DaysToReport (DateReported - DateTimeOfAccident) -> to buckets -> to one_hot
def bucket_days_to_report(dtt_array):
    return pd.cut(
        dtt_array.squeeze(), 
        bins=[-np.inf, 80, 300, 500, np.inf],
        labels=["<=80", "80-300", "300-500", ">500"]
    ).astype(str).values.reshape(-1, 1)

days_to_report_bucketer = FunctionTransformer(bucket_days_to_report, validate=False)
days_to_report_encoder = OneHotEncoder(drop='first')
days_to_report_pipeline = Pipeline([
    ('bucketizer', days_to_report_bucketer),
    ('encoder', days_to_report_encoder)
])

# DaysWorkedPerWeek -> 1 if equals 5, 0 in any other case 
def days_worked_binarize(days_array):
    # Ensure we handle arrays or DataFrames by squeezing to 1D
    days = days_array.squeeze()
    binarized = (days == 5).astype(int)
    # Return as 2D array: (n_samples x 1)
    return binarized.values.reshape(-1, 1) if isinstance(days, pd.Series) else binarized.reshape(-1, 1)

days_worked_transformer = FunctionTransformer(days_worked_binarize, validate=False)

In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ('log_inc', log_transformer, ['InitialIncurredCalimsCost', 'WeeklyWages']),
        ('minmax_scaler', MinMaxScaler(), ['Age', 'YearAccident']),
        ('gender_bool', gender_transformer, ['Gender']),
        ('hww_bool_onehot', hours_worked_pipeline, ['HoursWorkedPerWeek']),
        ('dtt_bool_onehot', days_to_report_pipeline, ['DaysToReport']),
        ('has_dependent_bool', Binarizer(threshold=0), ['DependentChildren']),
        ('worked_five_days_bool', days_worked_transformer, ['DaysWorkedPerWeek']),
        ('onehot', OneHotEncoder(drop='first'), ['MaritalStatus', 'PartTimeFullTime']),
    ],
    remainder='drop'
)

In [ ]:
### data fixes 

# m_weeklywages_under26 = df_train.WeeklyWages <= 26
# df_train.loc[m_weeklywages_under26, 'WeeklyWages'] = df_train.loc[~m_weeklywages_under26, 'WeeklyWages'].median()


# m_initial_claim_under10 = df_train.InitialIncurredCalimsCost <= 10
#df_train.loc[m_initial_claim_under10, 'InitialIncurredCalimsCost'] = df_train.loc[~m_initial_claim_under10, 'InitialIncurredCalimsCost'].median()


In [6]:
# Example:
X_train = df_train[['InitialIncurredCalimsCost', 'Age', 'Gender', 'DependentChildren', 'MaritalStatus', 
                    'WeeklyWages', 'PartTimeFullTime', 'HoursWorkedPerWeek',
                    'DaysWorkedPerWeek', 'YearAccident', 'DaysToReport']]
y_train = df_train['UltimateIncurredClaimCost']


In [ ]:
def run_experiment(
        experiment_name, 
        run_name, 
        regressor_object = LinearRegression(), 
        kfold = 5, 
        save_model=True
    ):

    regressor_pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('linear_model', regressor_object)
    ])

    model = TransformedTargetRegressor(
        regressor=regressor_pipeline,
        transformer=PowerTransformer(method='box-cox', standardize=False)
    )


    mlflow.set_experiment(experiment_name)

    with mlflow.start_run(run_name=run_name):
        # 1) Log relevant hyperparameters or pipeline details
        params = model.get_params(deep=True)
        for param_name, param_value in params.items():
            if isinstance(param_value, (str, int, float, bool, type(None))):
                mlflow.log_param(param_name, param_value)
            else:
                # Convert more complex objects to string
                mlflow.log_param(param_name, str(param_value))
        
        mlflow.log_param("n_features_in", X_train.shape[1])

        # Perform cross-validation with predictions
        y_pred = cross_val_predict(model, X_train, y_train, cv=kfold)

        # Compute overall errors
        overall_mse = mean_squared_error(y_train, y_pred)
        overall_rmse = np.sqrt(overall_mse)
        overall_mae = mean_absolute_error(y_train, y_pred)

        # Compute errors for y_true > 100,000
        high_mask = y_train > 100000
        if high_mask.sum() > 0:  # Ensure there are samples
            high_mse = mean_squared_error(y_train[high_mask], y_pred[high_mask])
            high_rmse = np.sqrt(high_mse)
            high_mae = mean_absolute_error(y_train[high_mask], y_pred[high_mask])
        else:
            high_mse, high_rmse, high_mae = np.nan, np.nan, np.nan  # No high-value samples

        # Compute errors for y_true ≤ 100,000
        low_mask = y_train <= 100000
        if low_mask.sum() > 0:  # Ensure there are samples
            low_mse = mean_squared_error(y_train[low_mask], y_pred[low_mask])
            low_rmse = np.sqrt(low_mse)
            low_mae = mean_absolute_error(y_train[low_mask], y_pred[low_mask])
        else:
            low_mse, low_rmse, low_mae = np.nan, np.nan, np.nan  # No low-value samples


        
        # 2) Perform cross-val
        scores = cross_val_score(
            model, 
            X_train, 
            y_train, 
            cv=kfold, 
            scoring='neg_mean_squared_error'
        )
        mse_scores = -scores
        rmse_scores = np.sqrt(mse_scores)

        print(f"CV MSE:  {mse_scores.mean():.3f}  (+/- {mse_scores.std():.3f})")
        print(f"CV RMSE: {rmse_scores.mean():.3f}  (+/- {rmse_scores.std():.3f})")

        # 3) Log metrics
        mlflow.log_metric("cv_mse", mse_scores.mean())
        mlflow.log_metric("cv_rmse", rmse_scores.mean())

        # 4) Fit the final model on the full train set
        # 5) Log the fitted pipeline as an MLflow artifact
        if save_model:
            model.fit(X_train, y_train)
            mlflow.sklearn.log_model(model, artifact_path="models")




In [8]:
n_vars = X_train.shape[1]

run_experiment(
    'Actuarial Loss Prediction - initial modelling',
    f'Lasso - {n_vars} var',
    Lasso()
)

run_experiment(
    'Actuarial Loss Prediction - initial modelling',
    f'XGBoost - {n_vars} var',
    XGBRegressor()
)

run_experiment(
    'Actuarial Loss Prediction - initial modelling',
    f'LightGBM - {n_vars} var',
    LGBMRegressor()
)

CV MSE:  1176452243.961  (+/- 533261087.004)
CV RMSE: 33602.085  (+/- 6881.287)


2025/02/25 09:51:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


CV MSE:  867841643.297  (+/- 529464483.964)
CV RMSE: 28424.890  (+/- 7737.393)


2025/02/25 09:51:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001471 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 617
[LightGBM] [Info] Number of data points in the train set: 43200, number of used features: 16
[LightGBM] [Info] Start training from score 5.903592


...

/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001307 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 616
[LightGBM] [Info] Number of data points in the train set: 43200, number of used features: 16
[LightGBM] [Info] Start training from score 5.933814


/Users/alexis/Projects/202502 Actuarial Loss Prediction/.env/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


CV MSE:  866981745.801  (+/- 529081345.265)
CV RMSE: 28410.596  (+/- 7734.326)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 617
[LightGBM] [Info] Number of data points in the train set: 54000, number of used features: 16
[LightGBM] [Info] Start training from score 5.928804


2025/02/25 09:52:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
n_vars = X_train.shape[1]

run_experiment(
    'Actuarial Loss Prediction - initial modelling - datafix',
    f'Lasso - {n_vars} var - weeklywages, initialclaim NO FIX',
    Lasso()
)

run_experiment(
    'Actuarial Loss Prediction - initial modelling - datafix',
    f'DT - {n_vars} var - weeklywages, initialclaim NO FIX',
    DecisionTreeRegressor()
)

run_experiment(
    'Actuarial Loss Prediction - initial modelling - datafix',
    f'RF - {n_vars} var - weeklywages, initialclaim NO FIX',
    RandomForestRegressor()
)

run_experiment(
    'Actuarial Loss Prediction - initial modelling - datafix',
    f'GB - {n_vars} var - weeklywages, initialclaim NO FIX',
    GradientBoostingRegressor()
)


Initial solution

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_squared_error
import scipy as sp
import numpy as np
from matplotlib import pyplot as plt

I'll run a first model using only the InitialIncurredCalimsCost, that will work as a baseline

In [ ]:
boxcox_transformer = PowerTransformer(method="box-cox", standardize=False)
y_train_transformed = boxcox_transformer.fit_transform(df_train[['UltimateIncurredClaimCost']])


In [ ]:
def inverse_boxcox(y_transformed, lmbda): 
    y_mean = np.mean(y_transformed)
    y_std = np.std(y_transformed)

    y_unscaled = (y_transformed * y_std) + y_mean

    return sp.special.inv_boxcox(y_unscaled, lmbda)

In [ ]:
lr = LinearRegression()
lr.fit(
    np.log(df_train[['InitialIncurredCalimsCost']]), 
    y_train_transformed
)

In [ ]:
y_pred_transformed = lr.predict(np.log(df_train[['InitialIncurredCalimsCost']]))

In [ ]:
y_pred = sp.special.inv_boxcox(y_pred_transformed, boxcox_transformer.lambdas_[0]).flatten()

In [ ]:
y_pred

In [ ]:
lr.coef_

In [ ]:
mean_squared_error(df_train['UltimateIncurredClaimCost'], y_pred)

In [ ]:
y_pred2 = inverse_boxcox(y_pred_transformed, boxcox_transformer.lambdas_[0])

In [ ]:
y_pred2.flatten()

In [ ]:
mean_squared_error(df_train['UltimateIncurredClaimCost'], y_pred2)